# 2nd Course, Part 2: Matching digit

We now turn to the second task and build a model that

- receives two images of hand-written digits as input and
- outputs a probability that both images show the same digit.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')


## Step 1: Preparing the data

We use the MNIST dataset again:

In [2]:
import tensorflow_datasets as tdfs

tdfs.disable_progress_bar()

mnist_train = tdfs.load(name='mnist', split='train')
mnist_test = tdfs.load(name='mnist', split='test')

mnist_train, mnist_test

(<DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 <DatasetV1Adapter shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>)

We now take pairs of subsequent samples, scale the images as before, and check whether the labels coincide:

In [6]:
mnist_train.batch(2)

<DatasetV1Adapter shapes: {image: (None, 28, 28, 1), label: (None,)}, types: {image: tf.uint8, label: tf.int64}>

In [7]:
def match_pairs(samples):
    images, digits = samples['image'], samples['label']
    matching = 1 if digits[0] == digits[1] else 0
    return (images[0] / 255, images[1]/255), matching


Xy_train = mnist_train.batch(2).map(match_pairs)
Xy_test = mnist_test.batch(2).map(match_pairs)

Xy_train

<DatasetV1Adapter shapes: (((28, 28, 1), (28, 28, 1)), ()), types: ((tf.float32, tf.float32), tf.int32)>

Let us see how many matching samples we have in our training set:


In [8]:
Xy_train.reduce(tf.constant((0,0)), lambda count, sample: count + (sample[1], 1))

<tf.Tensor: id=393, shape=(2,), dtype=int32, numpy=array([ 3026, 30000], dtype=int32)>

## Step 2: Building the model

The `Sequential` class allows us to conveniently construct a neural network by stacking layers.

But if we need more flexibility, for example, to construct

- a model with multiple inputs or multiple outputs, or
- a general directed acyclic graph of layers,

we need to use the `tf.keras.Model` class, also known as the functional API of keras.

The idea for our model is that we

1. apply our pre-trained digit-classifier to both images,
2. obtain two probability distributions $p^{(1)}$ and $p^{(2)}$,
3. and use a dense layer to deduce the desired probability.


In [14]:
CLASSIFIFER_PATH = 'classifier'

classifier = tf.keras.models.load_model(CLASSIFIFER_PATH)
classifier.trainable = False

def build_matcher_dense():
    image_1 = tf.keras.layers.Input((28,28,1))
    image_2 = tf.keras.layers.Input((28,28,1))
    probs_1 = classifier(image_1)
    probs_2 = classifier(image_2)
    both_probs = tf.keras.layers.Concatenate()([probs_1, probs_2])
    dense = tf.keras.layers.Dense(32, activation='relu')(both_probs)
    prediction = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    matcher = tf.keras.Model(inputs=[image_1, image_2], outputs=[prediction])
    return matcher


Let's train our matcher!

In [16]:
def train(model, nr_batches=400, nr_epochs=5):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(Xy_train.repeat().batch(32).take(nr_batches),
                        validation_data=Xy_test.repeat().batch(32).take(nr_batches // 2),
                        epochs=nr_epochs)
    
train(build_matcher_dense())

Epoch 1/5


      1/Unknown - 1s 632ms/step - loss: 0.7471 - accuracy: 0.3125

      2/Unknown - 1s 319ms/step - loss: 0.7315 - accuracy: 0.3438

      3/Unknown - 1s 214ms/step - loss: 0.7291 - accuracy: 0.3750

      4/Unknown - 1s 162ms/step - loss: 0.7248 - accuracy: 0.3750

...

362/400 [==========================>...] - ETA: 0s - loss: 0.0891 - accuracy: 0.9852

372/400 [==========================>...] - ETA: 0s - loss: 0.0889 - accuracy: 0.9851

383/400 [===========================>..] - ETA: 0s - loss: 0.0888 - accuracy: 0.9853

393/400 [============================>.] - ETA: 0s - loss: 0.0882 - accuracy: 0.9853

400/400 [==============================] - 3s 8ms/step - loss: 0.0883 - accuracy: 0.9852 - val_loss: 0.0738 - val_accuracy: 0.9886


## Step 3: Building a  model that need not learn

We now replace the last dense classification layer with a lambda layer that need not be trained, using the following observation:

Given the two probability distributions $p^{(1)}$ and $p^{(2)}$, the probability that both digits coincide is $\sum_i p^{(1)}_i p^{(2)}_i$.

We can implement the formula in 3. using a `Lambda` layer. The catch is that all
tensors flowing through the neural network are *batches* of data:

In [18]:
p1_batch = tf.constant([[0.1, 0.9], [0.5, 0.5]])
p2_batch = tf.constant([[0.2, 0.8],  [0.4, 0.6]])

def compute_prob_equality(p1_batch, p2_batch):
    return tf.reduce_sum(p1_batch * p2_batch, axis=-1)


compute_prob_equality(p1_batch, p2_batch)

<tf.Tensor: id=30389, shape=(2,), dtype=float32, numpy=array([0.73999995, 0.5       ], dtype=float32)>

In [17]:
def build_matcher_lambda():
    image_1 = tf.keras.layers.Input((28,28,1))
    image_2 = tf.keras.layers.Input((28,28,1))
    probs_1 = classifier(image_1)
    probs_2 = classifier(image_2)
    # both_probs = tf.keras.layers.Concatenate()([probs_1, probs_2])
    prediction = tf.keras.layers.Lambda(lambda p: tf.reduce_sum(p[0] * p[1], axis=-1, keepdims=True))([probs_1, probs_2])
    matcher = tf.keras.Model(inputs=[image_1, image_2], outputs=[prediction])
    return matcher

In [29]:
train(build_matcher_lambda())

Epoch 1/5


      1/Unknown - 0s 169ms/step - loss: 0.0265 - accuracy: 1.0000

      2/Unknown - 0s 89ms/step - loss: 0.0183 - accuracy: 1.0000 

      3/Unknown - 0s 61ms/step - loss: 0.0150 - accuracy: 1.0000

      4/Unknown - 0s 47ms/step - loss: 0.0123 - accuracy: 1.0000

...

365/400 [==========================>...] - ETA: 0s - loss: 0.0246 - accuracy: 0.9920

375/400 [===========================>..] - ETA: 0s - loss: 0.0250 - accuracy: 0.9918

385/400 [===========================>..] - ETA: 0s - loss: 0.0253 - accuracy: 0.9918

395/400 [============================>.] - ETA: 0s - loss: 0.0254 - accuracy: 0.9918

400/400 [==============================] - 3s 8ms/step - loss: 0.0254 - accuracy: 0.9917 - val_loss: 0.0295 - val_accuracy: 0.9905
